# ESMPy Tutorial
#### Nathan Wendt

ESMPy offers several features for taking data fields from one grid and interpolating it to another grid. Perhaps the easiest way is to load data in a SCRIP formatted file (most commonly seen in netCDF) into Python using this module. However, we do not always have our data formatted in this way. Many of us save our grids into less common formats (e.g., numpy compressed, etc.). What are we do do in these situations? Luckily, ESMPy has decent [documentation](http://www.earthsystemmodeling.org/esmf_releases/non_public/ESMF_7_0_0/esmpy_doc/html/index.html) and their email support is excellent.

Below is an example of how to do some common regridding tasks using ESMPy:

##### Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.basemap import Basemap
import ESMF
%matplotlib inline

Turn deubg output on.

In [ ]:
ESMF.Manager(debug=True)

##### Data

In [ ]:
# NAM
nam = np.load('nam_218_20120414_1200_006.npz')

# RUC
ruc = np.load('ruc2_130_20120414_1200_006.npz')

# SURFACE STATIONS
sfc = np.load('sfcstn_20120414_1800.npz')

# NAM CONUS MASK
nam_mask = np.load('nam_218_conus_mask.npz')

##### Grids

In [ ]:
nam_lat = nam['lat']
nam_lon = nam['lon']
nam_p4s = nam['p4s'].tolist()

In [ ]:
ruc_lat = ruc['lat']
ruc_lon = ruc['lon']
ruc_p4s = ruc['p4s'].tolist()

In [ ]:
mask = nam_mask['mask']

Let's see how these two grids compare.

In [ ]:
f1 = plt.figure(1,figsize=(16,9))
plt.scatter(nam_lon, nam_lat, color = cm.viridis.colors[0], label = 'NAM 218')
plt.scatter(ruc_lon, ruc_lat, color = cm.viridis.colors[164], label = 'RUC 130')
plt.legend()

## Interpolating From One Grid to Another

A common regridding task is to take numerical model output on one grid and interpolate it to another. For this example, we will take 2 m dewpoint temperatures from the NAM and regrid it to the RUC grid. The example data are from April 14, 2012.

##### Initialize Grids

We will need to create *Grid* objects for the two model grids we are using. Our **sourcegrid** will be from the **NAM** and our **destination** grid will be from the **RUC**. Our coordinates will be in the center of our gridpoints. As we are using latitude and longitude values, we will be using spherical coordinates.

In [ ]:
sourcegrid = ESMF.Grid(np.array(nam_lat.shape), staggerloc = ESMF.StaggerLoc.CENTER, coord_sys = ESMF.CoordSys.SPH_DEG)

In [ ]:
destgrid = ESMF.Grid(np.array(ruc_lat.shape), staggerloc = ESMF.StaggerLoc.CENTER, coord_sys = ESMF.CoordSys.SPH_DEG)

Right now, the *Grid* objects do not have any coordinates associated with them. We can add them easily.

In [ ]:
source_lon = sourcegrid.get_coords(0)
source_lat = sourcegrid.get_coords(1)

dest_lon = destgrid.get_coords(0)
dest_lat = destgrid.get_coords(1)

At this point we now have **pointers** to the _Grid_ object coordinates. Be careful not to overwrite the pointer variable here. You need to tell it to put the values into the array that is pointed to.

In [ ]:
source_lon[:] = nam_lon
source_lat[:] = nam_lat

dest_lon[:] = ruc_lon
dest_lat[:] = ruc_lat

##### Initialize Fields

Once the *Grid* objects are created, we need to add data (at least to the source grid). To do this, we use *Field* objects.

In [ ]:
sourcefield = ESMF.Field(sourcegrid, name = 'NAM 2 m DEWPOINT')

destfield = ESMF.Field(destgrid, name = 'RUC 2 m DEWPOINT')

Inserting the data is very simple. We will be using a 2 m dewpoint field (in Celsius) that was taken from the NAM grid.

In [ ]:
sourcefield.data[:] = nam['dpc']

##### Regrid

Now we need to initialize a *Regrid* object.

In [ ]:
regrid = ESMF.Regrid(sourcefield, destfield, regrid_method = ESMF.RegridMethod.BILINEAR, 
                     unmapped_action = ESMF.UnmappedAction.IGNORE)

Initializing the regrid object calculates the weights that will map on grid to another.

In [ ]:
destfield = regrid(sourcefield, destfield)

How does the interpolation compare to the original?

In [ ]:
# Set up the maps
nam_m = Basemap(llcrnrlon = nam_lon[0,0], llcrnrlat = nam_lat[0,0], urcrnrlon = nam_lon[-1,-1], 
                urcrnrlat = nam_lat[-1,-1], projection = nam_p4s['proj'], rsphere=(nam_p4s['a'], nam_p4s['b']),
                lat_0 = nam_p4s['lat_0'], lat_1 = nam_p4s['lat_1'], lat_2 = nam_p4s['lat_2'], 
                lon_0 = nam_p4s['lon_0'], resolution = 'l')

ruc_m = Basemap(llcrnrlon = ruc_lon[0,0], llcrnrlat = ruc_lat[0,0], urcrnrlon = ruc_lon[-1,-1], 
                urcrnrlat = ruc_lat[-1,-1], projection = ruc_p4s['proj'], rsphere=(ruc_p4s['a'], ruc_p4s['b']),
                lat_0 = ruc_p4s['lat_0'], lat_1 = ruc_p4s['lat_1'], lat_2 = ruc_p4s['lat_2'], 
                lon_0 = ruc_p4s['lon_0'], resolution = 'l')

In [ ]:
nam_x, nam_y = nam_m(nam_lon, nam_lat)
ruc_x, ruc_y = ruc_m(ruc_lon, ruc_lat)

In [ ]:
f2 = plt.figure(2, figsize=(16,9))

s1 = plt.subplot(121)
s1.set_title('NAM 218')
nam_m.pcolormesh(nam_x, nam_y, sourcefield.data, cmap = 'gist_ncar', vmax = 25, vmin = -25)
nam_m.drawstates()
nam_m.drawcountries()
nam_m.drawcoastlines()
ncb = nam_m.colorbar()
ncb.set_label('DWPT [C]')

s2 = plt.subplot(122)
s2.set_title('RUC 130')
ruc_m.pcolormesh(ruc_x, ruc_y, destfield.data, cmap = 'gist_ncar', vmax = 25, vmin = -25)
ruc_m.drawstates()
ruc_m.drawcountries()
ruc_m.drawcoastlines()
rcb = ruc_m.colorbar()
rcb.set_label('DWPT [C]')

## Interpolating Points to a Grid

Above, we loaded some surface station data that is not on any grid. We can regrid that to one of the model grids.

In [ ]:
sourceloc = ESMF.LocStream(len(sfc['dpc']), coord_sys = ESMF.CoordSys.SPH_DEG, 
                           name = 'Surface Station Dewpoint')

Now we will need to add the coordinates. In *LocStream* objects (unstructured points), these are accessed using dictionary keys.

In [ ]:
sourceloc['ESMF:Lat'] = sfc['lat']
sourceloc['ESMF:Lon'] = sfc['lon']

In this case, the surface data has some missing points. They are denoted by values of -9999. We can easily mask these values.

In [ ]:
# Where are the missing data?
missing = -9999
miss_loc = (sfc['dpc'] != missing).astype(np.int32)

# Define the mask in the LocStream
sourceloc['ESMF:Mask'] = miss_loc

In order to insert the data and do the regrid of the points, we need to make a *Field* object based on our *LocStream* object.

In [ ]:
sourcefield_pts = ESMF.Field(sourceloc, name = 'Surface Station Dewpoints')
sourcefield_pts.data[:] = sfc['dpc']

Because we now have data that does not extend beyond the CONUS, it would to add a mask to avoid undesired extrapolation. We can add a mask to the original NAM grid and use it here.

In [ ]:
sourcegrid_masked = sourcegrid.copy()
source_mask = sourcegrid_masked.add_item(ESMF.GridItem.MASK)
source_mask[:] = mask

Create a new *Grid* and *Regrid* objects and proceed as before.

In [ ]:
destfield2 = ESMF.Field(sourcegrid_masked, name = 'ASOS to NAM 218 Dewpoints')

# Initialize destination field to value outside data range for easy masking later
destfield2.data[:] = 1e20

As of right now, only **nearest neighbor** interpolation can be used when using *LocStream* objects for the source data.

In [ ]:
regrid2 = ESMF.Regrid(sourcefield_pts,destfield2, regrid_method = ESMF.RegridMethod.NEAREST_STOD, 
                      unmapped_action = ESMF.UnmappedAction.IGNORE, dst_mask_values=np.array([0,-9999]))

In [ ]:
# Use the ESMF.Region.SELECT to leave unmapped points as their current values (which will be 1e20)
destfield2 = regrid2(sourcefield_pts, destfield2, zero_region = ESMF.Region.SELECT)

How do the gridded surface observations compare to the model forecast field?

In [ ]:
# First, some masking voodoo for comparison purposes
nam_masked = np.ma.array(sourcefield.data)
nam_masked.mask = ~np.ma.make_mask(mask)

# Because we initially set the field to 1e20, we can 
# now use that to mask outside the CONUS easily without
# worrying about taking out real data
asos_masked = np.ma.masked_equal(destfield2.data, 1e20)
asos_masked = np.ma.masked_equal(asos_masked, -9999.0)

In [ ]:
f3 = plt.figure(3, figsize=(16,9))

s1 = plt.subplot(121)
s1.set_title('ASOS 2 M DWPT 041412/1800')
nam_m.pcolormesh(nam_x, nam_y, asos_masked, cmap = 'gist_ncar', vmax = 25, vmin = -25)
nam_m.drawstates()
nam_m.drawcountries()
nam_m.drawcoastlines()
ncb = nam_m.colorbar()
ncb.set_label('DWPT [C]')

s2 = plt.subplot(122)
s2.set_title('NAM 2 M DWPT 041412/1800')
nam_m.pcolormesh(nam_x, nam_y, nam_masked, cmap = 'gist_ncar', vmax = 25, vmin = -25)
nam_m.drawstates()
nam_m.drawcountries()
nam_m.drawcoastlines()
rcb = ruc_m.colorbar()
rcb.set_label('DWPT [C]')

And in a difference plot...

In [ ]:
f4 = plt.figure(4, figsize=(16,9))
plt.title('NAM - ASOS DWPT 041412/1800')
nam_m.pcolormesh(nam_x, nam_y, nam_masked - asos_masked, cmap = 'bwr', vmin = -25, vmax = 25)
nam_m.drawstates()
nam_m.drawcountries()
nam_m.drawcoastlines()
ncb = nam_m.colorbar()
ncb.set_label('DWPT DIFF [C]')

## Grid to LocStream

Another useful operation we can do with ESMPy is to pull values from specific points. Let's set up a few coordinates for some cities.

In [ ]:
marshall = (43.171111, -89.064722)
lacrosse = (43.813333, -91.233056)
lawrence = (38.866667, -95.233333)
urbana = (40.109665, -88.204247)
norman = (35.22, -97.44)

In [ ]:
town_lat = np.array([marshall[0], lacrosse[0], lawrence[0], urbana[0], norman[0]])
town_lon = np.array([marshall[1], lacrosse[1], lawrence[1], urbana[1], norman[1]])

Create the *LocStream* and populate the coordinates.

In [ ]:
town_loc = ESMF.LocStream(5, coord_sys = ESMF.CoordSys.SPH_DEG, name = 'Town Locations')

In [ ]:
town_loc['ESMF:Lat'] = town_lat
town_loc['ESMF:Lon'] = town_lon

Again, create a *Field* based on the town *LocStream*.

In [ ]:
town_data = ESMF.Field(town_loc, name = 'Town 2 m NAM Dewpoint')

Regrid using the NAM 2 M dewpoint data that we have from earlier.

In [ ]:
regrid3 = ESMF.Regrid(sourcefield,town_data, regrid_method = ESMF.RegridMethod.BILINEAR, 
                      unmapped_action = ESMF.UnmappedAction.IGNORE)

In [ ]:
town_data = regrid3(sourcefield, town_data)

And display the data.

In [ ]:
print("""
--- Extracted NAM 2 m Dewpoints ---
Marshall, WI   : {:5.3f} C
La Crosse, WI  : {:5.3f} C
Lawrence, KS   : {:5.3f} C
Urbana, IL     : {:5.3f} C
Norman, OK     : {:5.3f} C
""".format(*town_data.data))

## Other Features

ESMPy also has other capabilities not shown here. For example,
+ Regrid to and from meshes (think MPAS!)
+ Regrid in parallel
+ Read from files (netCDF, etc.) and automatically set up coordinates